In [46]:
import pandas as pd
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score

In [15]:
file_path ='itmo_sber.csv'
data = pd.read_csv('itmo_sber.csv')

In [17]:
print(data.isnull().sum())

Unnamed: 0                    0
Задача                        0
Обстановка                    0
Оптимальный план              0
Задача en                     0
Обстановка en                 0
Оптимальный план en           0
Предсказанный план            0
Успех предсказанного плана    0
dtype: int64


In [18]:
print(data.head())

   Unnamed: 0                          Задача  \
0           0   Приготовить горячий шоколад.    
1           1     Заказать доставку еды домой   
2           2  Реставрация старого комода\r\n   
3           3            Приготовить сырники.   
4           4               Нарезать колбасу.   

                                          Обстановка  \
0  кухня, молоко, какао, сахар, кастрюля, ложка, ...   
1  мобильный телефон, ресторанное меню, интернет,...   
2  Старый комод, повреждения, шпатель, шкурка, ма...   
3  Кухня, столешница, творог, мука, яйцо, сахар, ...   
4  Кухня, колбаса, нож, доска для резки, холодиль...   

                                    Оптимальный план  \
0  Откройте дверь. Зайдите на кухню. Закрйоте две...   
1  Найдите стол. Подойдите к столу. Найдите мобил...   
2  Подойдите к комоду.Найдите шкурку.Возьмите шку...   
3  Подойти к столешнице. Найти творог. Взять твор...   
4  Подойти к холодильнику. Открыть холодильник. Н...   

                               

In [19]:
print(data['Успех предсказанного плана'].value_counts())

Успех предсказанного плана
1    3234
0    1600
Name: count, dtype: int64


In [20]:
print(data.describe())

        Unnamed: 0  Успех предсказанного плана
count  4834.000000                 4834.000000
mean   2416.500000                    0.669011
std    1395.599931                    0.470618
min       0.000000                    0.000000
25%    1208.250000                    0.000000
50%    2416.500000                    1.000000
75%    3624.750000                    1.000000
max    4833.000000                    1.000000


In [21]:
from sklearn.feature_extraction.text import CountVectorizer

In [22]:
vectorizer = CountVectorizer()
X = vectorizer.fit_transform(data['Задача'])

In [23]:
print(X.shape)

(4834, 5402)


In [ ]:
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer

nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('omw-1.4')
nltk.download('punkt')

lemmatizer = WordNetLemmatizer()
stop_words = set(stopwords.words('russian'))

def preprocess_text(text):
    words = nltk.word_tokenize(text)
    words = [lemmatizer.lemmatize(word.lower()) for word in words if word.lower() not in stop_words and word.isalpha()]
    return ' '.join(words)

data['Задача'] = data['Задача'].apply(preprocess_text)
data['Обстановка'] = data['Обстановка'].apply(preprocess_text)

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\kiril\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\kiril\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\kiril\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\kiril\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [11]:
# # Инициализация векторизатора
# tfidf_vectorizer = TfidfVectorizer(max_features=5000)
#
# # Объединяем текстовые данные для векторизации
# combined_text = data['Задача'] + " " + data['Обстановка']
# X = tfidf_vectorizer.fit_transform(combined_text)
#
# # Целевая переменная
# y = data['Успех предсказанного плана']

In [12]:
# from sklearn.model_selection import train_test_split
# from sklearn.feature_extraction.text import TfidfVectorizer
# import re
#
# # Объединение текстовых полей
# data['combined_text'] = data['Задача en'] + ' ' + data['Обстановка en'] + ' ' + data['Предсказанный план']
#
# # Функция для очистки и нормализации текста
# def clean_text(text):
#     text = text.lower()  # Приведение к нижнему регистру
#     text = re.sub(r'\s+', ' ', text)  # Удаление лишних пробелов
#     text = re.sub(r'[^\w\s]', '', text)  # Удаление специальных символов
#     return text
#
# # Применение функции очистки к каждой строке объединенного текстового поля
# data['cleaned_text'] = data['combined_text'].apply(clean_text)
#
# # Подготовка данных и целевой переменной
# X = data['cleaned_text']
# y = data['Успех предсказанного плана']
#
# # Разделение данных на обучающую и тестовую выборки
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
#
# # Векторизация текста с использованием TF-IDF
# tfidf_vectorizer = TfidfVectorizer(max_features=1000)  # Ограничение на количество признаков для упрощения модели
# X_train_tfidf = tfidf_vectorizer.fit_transform(X_train)
# X_test_tfidf = tfidf_vectorizer.transform(X_test)
#
# X_train_tfidf.shape, X_test_tfidf.shape


In [25]:
from transformers import BertTokenizer, BertModel
import torch
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
import numpy as np

# Загрузка предобученной модели BERT и токенизатора
tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased')
model = BertModel.from_pretrained('bert-base-multilingual-cased')

print('AAA')
# Предварительная обработка текста
def preprocess_text(text):
    text = text.lower()
    text = text.strip()
    return text

data['Предсказанный план'] = data['Предсказанный план'].apply(preprocess_text)
data['Оптимальный план'] = data['Оптимальный план'].apply(preprocess_text)

n=0
# Функция для получения эмбеддингов текста с помощью BERT
def get_bert_embeddings(text):
    inputs = tokenizer(text, return_tensors="pt", padding=True, truncation=True)
    outputs = model(**inputs)
    embeddings = outputs.last_hidden_state.mean(dim=1).squeeze().detach().numpy()
    global n
    n=n+1
    if n%100==0:
        print(n)
    return embeddings

# Получение эмбеддингов для "Предсказанного плана" и "Оптимального плана"
predicted_plan_embeddings = np.array([get_bert_embeddings(text) for text in data['Предсказанный план']])
optimal_plan_embeddings = np.array([get_bert_embeddings(text) for text in data['Оптимальный план']])

# Вычисление сходства текстов с помощью косинусного сходства
cosine_similarities = [np.dot(pred, opt.T) / (np.linalg.norm(pred) * np.linalg.norm(opt)) for pred, opt in zip(predicted_plan_embeddings, optimal_plan_embeddings)]

# Подготовка данных и целевой переменной для классификации
X = np.array(cosine_similarities).reshape(-1, 1)
y = data['Успех предсказанного плана']

# Разделение на обучающую и тестовую выборки
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)


AAA
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000
3100
3200
3300
3400
3500
3600
3700
3800
3900
4000
4100
4200
4300
4400
4500
4600
4700
4800
4900
5000
5100
5200
5300
5400
5500
5600
5700
5800
5900
6000
6100
6200
6300
6400
6500
6600
6700
6800
6900
7000
7100
7200
7300
7400
7500
7600
7700
7800
7900
8000
8100
8200
8300
8400
8500
8600
8700
8800
8900
9000
9100
9200
9300
9400
9500
9600


# SMOTE

In [26]:
# Применение SMOTE для балансировки классов
from imblearn.over_sampling import SMOTE
smote = SMOTE(random_state=42)
X_train_resampled, y_train_resampled = smote.fit_resample(X_train, y_train)

In [27]:
print(X_train_resampled.shape)

(5180, 1)


In [47]:
import numpy as np

# Посчитаем количество элементов каждого класса в y_train_resampled
unique, counts = np.unique(y_train_resampled, return_counts=True)
class_counts = dict(zip(unique, counts))

print("Баланс классов в X_train_resampled:")
print(class_counts)


Баланс классов в X_train_resampled:
{0: 2590, 1: 2590}


In [48]:
X_train = X_train_resampled

In [49]:
y_train = y_train_resampled

# learn

In [50]:
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score

# Обучение классификатора
model = LogisticRegression()
model.fit(X_train, y_train)

# Предсказание на тестовой выборке
y_pred = model.predict(X_test)

# Оценка точности модели
accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy of the model: {accuracy}')

# Вычисление F1-меры
f1 = f1_score(y_test, y_pred)
print(f"F1 Score: {f1:.2f}")

Accuracy of the model: 0.5387797311271976
F1 Score: 0.62


In [51]:
# Посчитаем количество элементов каждого класса в y_train_resampled
unique, counts = np.unique(y_pred, return_counts=True)
class_counts = dict(zip(unique, counts))

print("Баланс классов в X_train_resampled:")
print(class_counts)

Баланс классов в X_train_resampled:
{0: 433, 1: 534}


# OPTUNA log reg

In [52]:
import optuna
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score

def objective(trial):
    # Определите пространство поиска гиперпараметров
    C = trial.suggest_float("C", 0.0001, 10000)
    penalty = trial.suggest_categorical("penalty", ["l2", "none"])
    max_iter = trial.suggest_int("max_iter", 100, 1000)

    # Модель логистической регрессии
    model = LogisticRegression(
        C=C,
        penalty=penalty,
        max_iter=max_iter
    )

    # Обучение
    model.fit(X_train, y_train)

    # Предсказание
    y_pred = model.predict(X_test)

    accuracy = accuracy_score(y_test, y_pred)

    return accuracy

study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=200)

#лучшие гиперпараметры
best_params = study.best_params
best_C = best_params["C"]
best_penalty = best_params["penalty"]
best_max_iter = best_params["max_iter"]

#модель с лучшими гиперпараметрами
best_model = LogisticRegression(
    C=best_C,
    penalty=best_penalty,
    max_iter=best_max_iter
)


best_model.fit(X, y)

y_pred=best_model.predict(X_test)

[I 2024-04-04 21:02:13,524] A new study created in memory with name: no-name-d2fe60d7-ad4f-448c-80db-ff1c9a45d6fb
[I 2024-04-04 21:02:13,540] Trial 0 finished with value: 0.5387797311271976 and parameters: {'C': 228.767833837904, 'penalty': 'l2', 'max_iter': 463}. Best is trial 0 with value: 0.5387797311271976.
[I 2024-04-04 21:02:13,555] Trial 1 finished with value: 0.5387797311271976 and parameters: {'C': 8435.539216557641, 'penalty': 'l2', 'max_iter': 394}. Best is trial 0 with value: 0.5387797311271976.
C:\Users\kiril\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\linear_model\_logistic.py:1183: FutureWarning: `penalty='none'`has been deprecated in 1.2 and will be removed in 1.4. To keep the past behaviour, set `penalty=None`.
  warnings.warn(
C:\Users\kiril\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\linear_model\_logistic.py:1193: UserWarning: Setting penalty=None will ignore the C and l1_ratio parameters
  warnings.warn(
[I 2024-04-04 21:

In [53]:
#Лучшие гиперпараметры для логистической регрессии
print(best_params)

# Оценим качество
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy:.2f}")

# Вычисление F1-меры
f1 = f1_score(y_test, y_pred)
print(f"F1 Score: {f1:.2f}")

# Вычисление ROC-AUC
roc_auc = roc_auc_score(y_test, y_pred)
print(f"ROC-AUC Score: {roc_auc:.2f}")

{'C': 228.767833837904, 'penalty': 'l2', 'max_iter': 463}
Accuracy: 0.67
F1 Score: 0.80
ROC-AUC Score: 0.50


In [38]:
# Посчитаем количество элементов каждого класса в y_train_resampled
unique, counts = np.unique(y_test, return_counts=True)
class_counts = dict(zip(unique, counts))

print("Баланс классов в X_train_resampled:")
print(class_counts)

Баланс классов в X_train_resampled:
{0: 323, 1: 644}


# ДЕРЕВЬЯ

In [29]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
import optuna

# Функция для оптимизации гиперпараметров модели Decision Tree
def objective_dt(trial):
    criterion = trial.suggest_categorical("criterion", ["gini", "entropy"])
    max_depth = trial.suggest_int("max_depth", 2, 32)
    min_samples_split = trial.suggest_int("min_samples_split", 2, 20)
    min_samples_leaf = trial.suggest_int("min_samples_leaf", 1, 20)

    model = DecisionTreeClassifier(
        criterion=criterion,
        max_depth=max_depth,
        min_samples_split=min_samples_split,
        min_samples_leaf=min_samples_leaf
    )

    model.fit(X_train, y_train)

    y_pred = model.predict(X_test)

    f1 = f1_score(y_test, y_pred)

    return f1

# Функция для оптимизации гиперпараметров модели Random Forest
def objective_rf(trial):
    n_estimators = trial.suggest_int("n_estimators", 50, 200)
    criterion = trial.suggest_categorical("criterion", ["gini", "entropy"])
    max_depth = trial.suggest_int("max_depth", 2, 32)
    min_samples_split = trial.suggest_int("min_samples_split", 2, 20)
    min_samples_leaf = trial.suggest_int("min_samples_leaf", 1, 20)

    model = RandomForestClassifier(
        n_estimators=n_estimators,
        criterion=criterion,
        max_depth=max_depth,
        min_samples_split=min_samples_split,
        min_samples_leaf=min_samples_leaf
    )

    model.fit(X_train, y_train)

    y_pred = model.predict(X_test)

    f1 = f1_score(y_test, y_pred)

    return f1

# Оптимизация гиперпараметров для модели Decision Tree
study_dt = optuna.create_study(direction="maximize")
study_dt.optimize(objective_dt, n_trials=100)

# Оптимизация гиперпараметров для модели Random Forest
study_rf = optuna.create_study(direction="maximize")
study_rf.optimize(objective_rf, n_trials=100)

# Получение лучших гиперпараметров для Decision Tree
best_params_dt = study_dt.best_params

# Получение лучших гиперпараметров для Random Forest
best_params_rf = study_rf.best_params

# Создание моделей с лучшими гиперпараметрами
best_model_dt = DecisionTreeClassifier(**best_params_dt)
best_model_rf = RandomForestClassifier(**best_params_rf)

# Обучение моделей
best_model_dt.fit(X_train, y_train)
best_model_rf.fit(X_train, y_train)

# Оценка качества моделей
y_pred_dt = best_model_dt.predict(X_test)
y_pred_rf = best_model_rf.predict(X_test)

f1_dt = f1_score(y_test, y_pred_dt)
f1_rf = f1_score(y_test, y_pred_rf)


print(f"F1 Score for Decision Tree: {f1_dt:.2f}")
print(f"F1 Score for Random Forest: {f1_rf:.2f}")

accuracy_dt = accuracy_score(y_test, y_pred_dt)
accuracy_rf = accuracy_score(y_test, y_pred_rf)
print(f"accuracy_dt: {accuracy_dt:.2f}")
print(f"accuracy_rf: {accuracy_rf:.2f}")

[I 2024-04-04 15:40:41,997] A new study created in memory with name: no-name-fcc86695-7e92-4c5b-8b3f-c9408feebe7b
[I 2024-04-04 15:40:42,015] Trial 0 finished with value: 0.7010752688172043 and parameters: {'criterion': 'gini', 'max_depth': 21, 'min_samples_split': 4, 'min_samples_leaf': 15}. Best is trial 0 with value: 0.7010752688172043.
[I 2024-04-04 15:40:42,030] Trial 1 finished with value: 0.7598425196850394 and parameters: {'criterion': 'entropy', 'max_depth': 10, 'min_samples_split': 16, 'min_samples_leaf': 9}. Best is trial 1 with value: 0.7598425196850394.
[I 2024-04-04 15:40:42,064] Trial 2 finished with value: 0.7516778523489934 and parameters: {'criterion': 'gini', 'max_depth': 11, 'min_samples_split': 16, 'min_samples_leaf': 11}. Best is trial 1 with value: 0.7598425196850394.
[I 2024-04-04 15:40:42,092] Trial 3 finished with value: 0.6968124536693847 and parameters: {'criterion': 'gini', 'max_depth': 25, 'min_samples_split': 2, 'min_samples_leaf': 3}. Best is trial 1 wit

F1 Score for Decision Tree: 0.80
F1 Score for Random Forest: 0.80
accuracy_dt: 0.66
accuracy_rf: 0.66


# catboost


In [55]:
from catboost import CatBoostClassifier
import optuna

# Определение функции для оптимизации гиперпараметров CatBoost
def objective_cb(trial):
    params = {
        "iterations": trial.suggest_int("iterations", 100, 1000),
        "learning_rate": trial.suggest_loguniform("learning_rate", 0.001, 0.1),
        "depth": trial.suggest_int("depth", 3, 10),
        "l2_leaf_reg": trial.suggest_loguniform("l2_leaf_reg", 1e-3, 10),
        "border_count": trial.suggest_int("border_count", 32, 255),
        "random_strength": trial.suggest_loguniform("random_strength", 1e-3, 10),
        "bagging_temperature": trial.suggest_loguniform("bagging_temperature", 1e-3, 10),
        "min_data_in_leaf": trial.suggest_int("min_data_in_leaf", 1, 20),
        "leaf_estimation_iterations": trial.suggest_int("leaf_estimation_iterations", 1, 10),
        "leaf_estimation_method": trial.suggest_categorical("leaf_estimation_method", ["Newton", "Gradient"]),
        "boosting_type": "Plain"  # Не стохастический бустинг для ускорения обучения
    }

    model = CatBoostClassifier(**params)
    model.fit(X_train, y_train, verbose=0)

    y_pred = model.predict(X_test)

    f1 = f1_score(y_test, y_pred)
    # roc_auc = roc_auc_score(y_test, y_pred)

    return f1
    # return roc_auc


# Оптимизация гиперпараметров для CatBoost
study_cb = optuna.create_study(direction="maximize")
study_cb.optimize(objective_cb, n_trials=50)

# Получение лучших гиперпараметров для CatBoost
best_params_cb = study_cb.best_params

# Создание модели с лучшими гиперпараметрами
best_model_cb = CatBoostClassifier(**best_params_cb)

# Обучение модели
best_model_cb.fit(X_train, y_train, verbose=0)

# Оценка качества модели
y_pred_cb = best_model_cb.predict(X_test)
f1_cb = f1_score(y_test, y_pred_cb)

print(f"F1 Score for CatBoost: {f1_cb:.2f}")
print(f"F1 Score for CatBoost: {f1_cb:.2f}")

accuracy = accuracy_score(y_test, y_pred_cb)
print(f"accuracy CatBoost:: {accuracy :.2f}")

[I 2024-04-04 21:05:48,058] A new study created in memory with name: no-name-e52c390d-14c0-41cd-b6db-3c521c1f0a69
C:\Users\kiril\AppData\Local\Temp\ipykernel_18140\3533341166.py:8: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  "learning_rate": trial.suggest_loguniform("learning_rate", 0.001, 0.1),
C:\Users\kiril\AppData\Local\Temp\ipykernel_18140\3533341166.py:10: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  "l2_leaf_reg": trial.suggest_loguniform("l2_leaf_reg", 1e-3, 10),
C:\Users\kiril\AppData\Local\Temp\ipykernel_18140\3533341166.py:12: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/opt

F1 Score for CatBoost: 0.62
F1 Score for CatBoost: 0.62
accuracy CatBoost:: 0.53


# АНСАМБЛИ

In [56]:
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, VotingClassifier
from sklearn.metrics import f1_score
import optuna

# Определение функций для оптимизации гиперпараметров случайного леса, градиентного бустинга и SVM
def objective_rf(trial):
    # Определение гиперпараметров для случайного леса
    n_estimators = trial.suggest_int("n_estimators", 50, 200)
    criterion = trial.suggest_categorical("criterion", ["gini", "entropy"])
    max_depth = trial.suggest_int("max_depth", 2, 32)
    min_samples_split = trial.suggest_int("min_samples_split", 2, 20)
    min_samples_leaf = trial.suggest_int("min_samples_leaf", 1, 20)

    model = RandomForestClassifier(
        n_estimators=n_estimators,
        criterion=criterion,
        max_depth=max_depth,
        min_samples_split=min_samples_split,
        min_samples_leaf=min_samples_leaf
    )

    model.fit(X_train, y_train)

    y_pred = model.predict(X_test)

    accuracy = accuracy_score(y_test, y_pred)

    return accuracy

def objective_gb(trial):
    # Определение гиперпараметров для градиентного бустинга
    n_estimators = trial.suggest_int("n_estimators", 50, 200)
    learning_rate = trial.suggest_loguniform("learning_rate", 0.001, 0.1)
    max_depth = trial.suggest_int("max_depth", 2, 32)
    min_samples_split = trial.suggest_int("min_samples_split", 2, 20)
    min_samples_leaf = trial.suggest_int("min_samples_leaf", 1, 20)

    model = GradientBoostingClassifier(
        n_estimators=n_estimators,
        learning_rate=learning_rate,
        max_depth=max_depth,
        min_samples_split=min_samples_split,
        min_samples_leaf=min_samples_leaf
    )

    model.fit(X_train, y_train)

    y_pred = model.predict(X_test)

    accuracy = accuracy_score(y_test, y_pred)

    return accuracy

def objective_svm(trial):
    # Определение гиперпараметров для SVM
    C = trial.suggest_float("C", 0.01, 10.0)
    kernel = trial.suggest_categorical("kernel", ["linear", "rbf", "poly", "sigmoid"])
    degree = trial.suggest_int("degree", 2, 5)

    model = SVC(
        C=C,
        kernel=kernel,
        degree=degree,
        probability=True
    )

    model.fit(X_train, y_train)

    y_pred = model.predict(X_test)

    accuracy = accuracy_score(y_test, y_pred)

    return accuracy

# Оптимизация гиперпараметров для случайного леса
study_rf = optuna.create_study(direction="maximize")
study_rf.optimize(objective_rf, n_trials=50)

# Оптимизация гиперпараметров для градиентного бустинга
study_gb = optuna.create_study(direction="maximize")
study_gb.optimize(objective_gb, n_trials=50)

# Оптимизация гиперпараметров для SVM
study_svm = optuna.create_study(direction="maximize")
study_svm.optimize(objective_svm, n_trials=2)

# Получение лучших гиперпараметров для случайного леса, градиентного бустинга и SVM
best_params_rf = study_rf.best_params
best_params_gb = study_gb.best_params
best_params_svm = study_svm.best_params

# Создание моделей с лучшими гиперпараметрами
best_model_rf = RandomForestClassifier(**best_params_rf)
best_model_gb = GradientBoostingClassifier(**best_params_gb)
best_model_svm = SVC(**best_params_svm, probability=True)  # probability=True для возможности использования soft voting

# Обучение моделей
best_model_rf.fit(X_train, y_train)
best_model_gb.fit(X_train, y_train)
best_model_svm.fit(X_train, y_train)

# Создание ансамбля с помощью голосования
voting_clf = VotingClassifier(
    estimators=[('rf', best_model_rf), ('gb', best_model_gb), ('svm', best_model_svm)],
    voting='soft'
)

# Обучение ансамбля
voting_clf.fit(X_train, y_train)

# Оценка качества ансамбля
y_pred_voting = voting_clf.predict(X_test)
f1_voting = f1_score(y_test, y_pred_voting)

accuracy_voting = accuracy_score(y_test, y_pred_voting)

print(f"F1 Score for Voting Classifier: {f1_voting:.2f}")
print(f"ACCURACY for Voting Classifier: {accuracy_voting:.2f}")


[I 2024-04-04 21:12:41,788] A new study created in memory with name: no-name-9d108f0d-921b-40fc-8bdb-deadfe60e81e
[I 2024-04-04 21:12:45,956] Trial 0 finished with value: 0.516028955532575 and parameters: {'n_estimators': 136, 'criterion': 'entropy', 'max_depth': 26, 'min_samples_split': 4, 'min_samples_leaf': 9}. Best is trial 0 with value: 0.516028955532575.
[I 2024-04-04 21:12:47,518] Trial 1 finished with value: 0.5274043433298863 and parameters: {'n_estimators': 69, 'criterion': 'gini', 'max_depth': 24, 'min_samples_split': 16, 'min_samples_leaf': 16}. Best is trial 1 with value: 0.5274043433298863.
[I 2024-04-04 21:12:50,898] Trial 2 finished with value: 0.5098241985522234 and parameters: {'n_estimators': 160, 'criterion': 'entropy', 'max_depth': 22, 'min_samples_split': 18, 'min_samples_leaf': 12}. Best is trial 1 with value: 0.5274043433298863.
[I 2024-04-04 21:12:51,439] Trial 3 finished with value: 0.4643226473629783 and parameters: {'n_estimators': 71, 'criterion': 'gini', '

F1 Score for Voting Classifier: 0.65
ACCURACY for Voting Classifier: 0.56


In [57]:
file_path = 'final_test_data.csv'
data = pd.read_csv('final_test_data.csv')
# Инициализация векторизатора
tfidf_vectorizer = TfidfVectorizer(max_features=5000)

# Объединяем текстовые данные для векторизации
combined_text = data['Задача en'] + " " + data['Обстановка en']
X2 = tfidf_vectorizer.fit_transform(combined_text)

# Целевая переменная
y2 = data['Успех предсказанного плана']

best_model.fit(X, y)

y_pred = best_model.predict(X2)
#Лучшие гиперпараметры для логистической регрессии
print(best_params)

# Оценим качество
accuracy = accuracy_score(y2, y_pred)
print(f"Accuracy: {accuracy:.2f}")

# Вычисление F1-меры
f1 = f1_score(y2, y_pred)
print(f"F1 Score: {f1:.2f}")

# Вычисление ROC-AUC
roc_auc = roc_auc_score(y2, y_pred)
print(f"ROC-AUC Score: {roc_auc:.2f}")

ValueError: X has 3011 features, but LogisticRegression is expecting 1 features as input.